In [19]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from pypdf import PdfReader
from langchain.chat_models import ChatOllama
from langchain.chains import RetrievalQA
import gradio as gr

In [8]:

reader = PdfReader("Options_Volatility_Script_Documentation.pdf")
text = ''

for page in reader.pages:
    text += page.extract_text()

print(text[:500])

Options Volatility Script: 
Documentation
Key Features:
Option Chain Data: Fetches live call/put options from NSE.
Historical Data: Retrieves daily OHLC data from NSE (last 90 days).
Historical Volatility HV Calculates past stock price fluctuations.
GARCH Volatility: Models future volatility based on past returns.
Implied Volatility IV Derives market's expectation of future volatility from 
option prices.
Excel Output: Saves all data into an organized Excel file.
Mathematical Formulas :
1.


In [14]:
raw_text = text

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

texts = text_splitter.split_text(raw_text)


embeddings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_texts( texts , embedding=embeddings)

vectorstore.save_local('volatility_vectordb')

In [16]:
embedding_model = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("volatility_vectordb", embeddings=embedding_model, allow_dangerous_deserialization=True)

llm = ChatOllama(model="llama2")

rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever = vectorstore.as_retriever(), return_source_documents=True)


query = "What is the document about"

result = rag_chain(query)

print(f"Answer: {result['result']}")


for doc in result['source_documents']:
    print('--- Source Chunk ---')
    print(doc.page_content[:300])

Answer: The document appears to be a user manual or documentation for an Options Volatility Script. The script provides various features and tools for calculating and analyzing options volatility, including:

1. Option Chain Data: The script can fetch live call/put options from NSE (National Stock Exchange of India).
2. Historical Data: The script can retrieve daily OHLC (Open, High, Low, Close) data from NSE for the last 90 days.
3. Historical Volatility: The script calculates past stock price fluctuations using a formula that is based on the annualized standard deviation of daily log returns.
4. GARCH Volatility: The script uses a conditional volatility modeling approach, accounting for volatility clustering, to estimate the volatility of the underlying asset.
5. Black-Scholes Option Pricing: The script uses the Black-Scholes model to price European options and find the implied volatility that makes the Black-Scholes price match the market price.
6. Implied Volatility: The script pro

In [23]:
def chat(query):
    try:
        print("Received Query:", query) 
        print("Generated Answer:", result)
        return rag_chain.invoke(query)["result"]
    except Exception as e:
        print("Error occurred:", str(e))
        return f"❌ Error: {str(e)}"

interface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=2, placeholder="ask about your doc"),
    outputs='text',
    title='[Rag + Ollama] PDF Q/A',
    theme='soft',
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Received Query: what does the document say
Generated Answer: {'query': 'What is the document about', 'result': 'The document appears to be a user manual or documentation for an Options Volatility Script. The script provides various features and tools for calculating and analyzing options volatility, including:\n\n1. Option Chain Data: The script can fetch live call/put options from NSE (National Stock Exchange of India).\n2. Historical Data: The script can retrieve daily OHLC (Open, High, Low, Close) data from NSE for the last 90 days.\n3. Historical Volatility: The script calculates past stock price fluctuations using a formula that is based on the annualized standard deviation of daily log returns.\n4. GARCH Volatility: The script uses a conditional volatility modeling approach, accounting for volatility clustering, to estimate the volatility of the underlying asset.\n5. Black-Scholes Option Pricing: The script uses the Black-Scholes model to price European options and find the impli